In [1]:
import visualize_wealth
import visualize_wealth.construct_portfolio as vwcp
import pandas

Let's use the `static_allocation` provided in the `panel from weight
file.xlsx` workbook

In [24]:
f = pandas.ExcelFile('test_data/panel from weight file test.xlsx')
static_alloc = f.parse('static_allocation_arsinha', index_col = 0, header_col = 0)

Again, assume we don't have the prices and need to donwload them, use
the `fetch_data_for_initial_allocation_method`

In [25]:
price_panel = vwcp.fetch_data_for_initial_allocation_method(static_alloc)

Construct the `panel` for the portoflio while determining the desired
rebalance frequency

In [26]:
panel = vwcp.panel_from_initial_weights(weight_series = static_alloc, price_panel = price_panel, rebal_frequency = 'quarterly')

Construct the final portfolio with columns `['Open', 'Close']`

In [27]:
portfolio = vwcp.pfp_from_weight_file(panel)

Take a look at the portfolio series:

In [28]:
portfolio.head()

,Close,Open
Date,,
2013-08-15,1000.000000,1001.691331
2013-08-16,997.035091,1002.110437
2013-08-19,987.847817,994.971873
2013-08-20,991.122052,987.937897
2013-08-21,981.312676,986.961156


In [29]:
portfolio

,Close,Open
Date,,
2013-08-15,1000.000000,1001.691331
2013-08-16,997.035091,1002.110437
2013-08-19,987.847817,994.971873
2013-08-20,991.122052,987.937897
2013-08-21,981.312676,986.961156
2013-08-22,989.168023,985.655602
2013-08-23,995.849765,992.785480
2013-08-26,990.686985,995.428831
2013-08-27,976.079787,981.089612


In [35]:
from datetime import date
a = date(2017,3,24)
b = date(2013,8,15)
(a-b).days/365

3.6082191780821917

In [37]:
ret_per_year=(1240.43-1000)/3.61

In [38]:
ret_per_year*100/1000

6.660110803324102